# 03 - Model Training

Entrenamiento y evaluación del modelo MarketValueNet.

Este notebook cubre:
- Carga de features procesadas
- Creación del modelo y DataLoaders
- Entrenamiento con early stopping
- Evaluación: métricas, confusion matrix, curvas de entrenamiento
- Feature importance

In [ ]:
import sys
sys.path.insert(0, '..')

import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

from src.model.architecture import MarketValueNet
from src.model.dataset import PolymarketDataset, create_dataloaders
from src.model.train import train_model
from src.model.evaluate import evaluate_model, print_evaluation

sns.set_theme(style='whitegrid')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')

## 1. Cargar datos procesados

In [ ]:
# Cargar dataset
dataset = PolymarketDataset.from_numpy_dir('../data/processed')

print(f'Dataset size: {len(dataset)}')
print(f'Numerical features shape: {dataset.numerical.shape}')
print(f'Text embeddings shape: {dataset.text_emb.shape}')
print(f'\nLabel distribution:')
print(f'  Buy (1):    {int(dataset.labels.sum())} ({dataset.labels.mean():.1%})')
print(f'  No Buy (0): {len(dataset) - int(dataset.labels.sum())} ({1 - dataset.labels.mean():.1%})')

In [ ]:
# Crear DataLoaders
train_loader, val_loader = create_dataloaders(
    dataset,
    batch_size=64,
    val_split=0.2,
    use_weighted_sampler=True
)

print(f'Train batches: {len(train_loader)}')
print(f'Val batches: {len(val_loader)}')

## 2. Crear y entrenar el modelo

In [ ]:
# Crear modelo
model = MarketValueNet(
    num_numerical_features=dataset.numerical.shape[1],
    num_categories=20,
    category_embed_dim=8,
    text_embed_dim=dataset.text_emb.shape[1],
    hidden_dims=[256, 128, 64],
    dropout=0.3,
    task='classification',
)

total_params = sum(p.numel() for p in model.parameters())
print(f'Model architecture:\n{model}')
print(f'\nTotal parameters: {total_params:,}')

In [ ]:
# Entrenar
history = train_model(
    model,
    train_loader,
    val_loader,
    epochs=50,
    lr=1e-3,
    device=device,
    save_dir='../data/models',
)

## 3. Curvas de entrenamiento

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train', linewidth=2)
axes[0].plot(history['val_loss'], label='Validation', linewidth=2)
axes[0].set_title('Loss por Época')
axes[0].set_xlabel('Época')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
if history.get('val_accuracy'):
    axes[1].plot(history['val_accuracy'], color='green', linewidth=2)
    axes[1].set_title('Accuracy de Validación')
    axes[1].set_xlabel('Época')
    axes[1].set_ylabel('Accuracy')
    axes[1].grid(True, alpha=0.3)

plt.suptitle('Curvas de Entrenamiento', fontsize=14)
plt.tight_layout()
plt.savefig('../figures/03_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Evaluación del modelo

In [ ]:
# Cargar mejor modelo
model.load_state_dict(torch.load('../data/models/best_market_model.pt', weights_only=True))

# Evaluar en validación
results = evaluate_model(model, val_loader, device=device)
print_evaluation(results)

In [ ]:
# Confusion Matrix
from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize=(7, 6))
cm = results['confusion_matrix']
disp = ConfusionMatrixDisplay(cm, display_labels=['No Buy', 'Buy'])
disp.plot(ax=ax, cmap='Blues', values_format='d')
ax.set_title('Confusion Matrix')
plt.tight_layout()
plt.savefig('../figures/03_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Distribución de scores del modelo
fig, ax = plt.subplots(figsize=(10, 6))

scores = results['scores']
labels_eval = results['labels']

ax.hist(scores[labels_eval == 0], bins=50, alpha=0.5, label='No Buy (real)', color='coral', density=True)
ax.hist(scores[labels_eval == 1], bins=50, alpha=0.5, label='Buy (real)', color='steelblue', density=True)
ax.axvline(x=0.5, color='black', linestyle='--', label='Threshold (0.5)')
ax.set_title('Distribución de Scores del Modelo')
ax.set_xlabel('Model Score')
ax.set_ylabel('Densidad')
ax.legend()
plt.tight_layout()
plt.savefig('../figures/03_score_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ROC Curve
from sklearn.metrics import roc_curve, auc

fpr, tpr, thresholds = roc_curve(labels_eval, scores)
roc_auc = auc(fpr, tpr)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fpr, tpr, color='steelblue', linewidth=2, label=f'ROC (AUC = {roc_auc:.3f})')
ax.plot([0, 1], [0, 1], color='gray', linestyle='--')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve')
ax.legend()
plt.tight_layout()
plt.savefig('../figures/03_roc_curve.png', dpi=150, bbox_inches='tight')
plt.show()